# LIBRARIES

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch import optim

import numpy as np

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DATASET

In [2]:

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)



# LINEAR NEURAL NET

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

#  CONVOLUTION NEURAL NETWORK

In [10]:
class ConvNet(nn.Module):
    def __init__(self, input_shape=(1,28,28)):
        super(ConvNet, self).__init__()

        # DEFINE THE CONVOLUTION LAYERS

        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64,3)
        self.conv3 = nn.Conv2d(64, 128, 3)

        # DEFINE THE POOLING LATER

        self.pool = nn.MaxPool2d(2, 2)

        n_size = self._get_conv_output(input_shape)

        # DEFINE THE LINEAR CLASSS

        self.fc1 = nn.Linear(n_size, 512)
        self.fc2 = nn.Linear(512, 10)

    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    def _forward_features(self, X):
        X = self.pool(F.relu(self.conv1(X)))
        X = self.pool(F.relu(self.conv2(X)))
        X = self.pool(F.relu(self.conv3(X)))
        return X

    def forward(self, X):
        X = self._forward_features(X)
        X = X.view(X.size(0), -1)
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        return X
model = ConvNet()
model.to(device)
print(model)

#loss = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters())



ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


# HYPERPARAMETERS

In [11]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# LOSS FUNCTION

In [12]:

loss_fn = nn.CrossEntropyLoss()

# OPTIMIZER

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# TRAINING

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>3f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Metrics: \n Accuracy: {(100*correct):>0.1f}%, Test loss: {test_loss:>8f} \n")

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302054  [    0/60000]
loss: 2.305269  [ 6400/60000]
loss: 2.300631  [12800/60000]
loss: 2.298966  [19200/60000]
loss: 2.299102  [25600/60000]
loss: 2.294599  [32000/60000]
loss: 2.297127  [38400/60000]
loss: 2.291424  [44800/60000]
loss: 2.293116  [51200/60000]
loss: 2.296901  [57600/60000]
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.293811 

Epoch 2
-------------------------------
loss: 2.294776  [    0/60000]
loss: 2.296739  [ 6400/60000]
loss: 2.291722  [12800/60000]
loss: 2.291606  [19200/60000]
loss: 2.292606  [25600/60000]
loss: 2.284849  [32000/60000]
loss: 2.289898  [38400/60000]
loss: 2.282740  [44800/60000]
loss: 2.284348  [51200/60000]
loss: 2.286742  [57600/60000]
Test Error: 
 Accuracy: 19.8%, Avg loss: 2.282802 

Epoch 3
-------------------------------
loss: 2.285632  [    0/60000]
loss: 2.286147  [ 6400/60000]
loss: 2.278996  [12800/60000]
loss: 2.279516  [19200/60000]
loss: 2.283044  [25600/60000]
loss: 2.268442  [32000/600